In [1]:
import tensorflow as tf
import apache_beam as beam

In [2]:
live_model = tf.keras.models.load_model('/Users/xxx/kagglepipe/titanic/pipeline/Trainer/model/6/serving_model_dir/')

In [4]:
with beam.Pipeline() as pipeline:
      test_data = pipeline | 'ReadFromTFRecord' >> beam.io.ReadFromTFRecord(file_pattern='/Users/xxx/kagglepipe/titanic/data/test/test.tfrecord')
      (test_data | 'ParseToExample' >> beam.Map(tf.train.Example.FromString)
      | 'Prediction' >> beam.Map(lambda x: {'id': x.features.feature['PassengerId'].int64_list.value[0], 'prediction': live_model.signatures["serving_default"](examples=tf.constant([x.SerializeToString()]))})
      | 'TensorToNumpy' >> beam.Map(lambda x: (x['id'], x['prediction']['output_0'].numpy()[0][0]))
      | 'FormatToStr' >> beam.Map(lambda x: '%s,%s' % (x[0], '1' if x[1] > 0.5  else '0'))
      | beam.io.WriteToText('/Users/xxx/kagglepipe/titanic/test.csv'))